# **1. Installing Libraries**

In [1]:
# Installs crewaI and crewaI-tools Python packages for agent-based workflows and tool integrations.
!pip install crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.7/602.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.8 MB/

# **2. Setting up Environment**

In [7]:
from crewai import Agent, Task, Crew, Process
import os
from crewai.knowledge.source.text_file_knowledge_source import TextFileKnowledgeSource

In [8]:
# Set your OpenAI API key as an environment variable so that the respective tools can access it.
# (NOTE: The API key is left empty for security; insert your own if running this code.)
os.environ['OPENAI_API_KEY'] = ""

# **3. Setting Up Tool for Image Generation**

In [9]:
import json
import uuid
import os
from typing import Type

from crewai.tools import BaseTool
from openai import OpenAI
from pydantic import BaseModel

# Define input validation schema for the DALL·E image generation tool.
# Ensures the prompt/description is a non-empty string.
class DalleImagePromptSchema(BaseModel):
    """Input schema for DALL·E Tool."""
    description: str = "Description of the image to be generated by the DALL·E model."

# Custom tool for image generation using OpenAI's DALL·E model.
class DalleImageTool(BaseTool):
    name: str = "DALL·E Image Tool"
    description: str = "Generates images using OpenAI's DALL·E model."
    args_schema: Type[BaseModel] = DalleImagePromptSchema

    model: str = "dall-e-3"
    size: str = "1024x1024"
    n: int = 1
    quality: str = "standard"  # or "hd" if available

    def _run(self, **kwargs) -> str:
        client = OpenAI()

        prompt: str = kwargs.get("description")
        if not prompt:
            return "Image description is required."

        response = client.images.generate(
            model=self.model,
            prompt=prompt,
            size=self.size,
            quality=self.quality,
            n=self.n,
            response_format="url"
        )

        image_url = response.data[0].url

        return json.dumps({
            "prompt": prompt,
            "image_url": image_url
        })


# **3. Setting Up Tool for Backstory Generation**

In [10]:
import os
from openai import OpenAI
from crewai.tools import BaseTool
from pydantic import BaseModel

# Input schema for the backstory generator tool (expects a refined prompt)
class BackstoryPromptSchema(BaseModel):
    """Schema for the backstory generator."""
    refined_prompt: str = "The refined avatar prompt or user description."

# Custom tool for generating character backstories using GPT-4
class BackstoryGeneratorTool(BaseTool):
    name: str = "Avatar Backstory Generator"
    description: str = (
        "Takes a refined avatar prompt and returns a rich, 200–300 word backstory "
        "covering origins, key events, personality, and cultural background."
    )
    args_schema: type[BaseModel] = BackstoryPromptSchema

    def _run(self, **kwargs) -> str:
        prompt = kwargs.get("refined_prompt")
        if not prompt:
            return "A refined prompt is required to generate a backstory."
        # system is how gpt should behave to write the backstory of generated character
        system = (
            "You are a master storyteller and world‑builder. "
            "Your task is to breathe life into characters by weaving together "
            "plot threads, cultural details, and emotional arcs."
        )
        # user tells  what user is expecting and what would output look like
        user = (
            f"Generate a vivid backstory (200–300 words) for an avatar described as:\n\n"
            f"{prompt}\n\n"
            "Cover:\n"
            "1. Origins and heritage\n"
            "2. Key life events or turning points\n"
            "3. Core personality traits and motivations\n"
            "4. Cultural influences or traditions\n"
            "Tie the narrative to visual elements implied by the prompt."
        )

        # Instantiate client inside the run method
        client = OpenAI()

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": user}
            ]
        )
        return response.choices[0].message.content

# **5. Agents Structure and setting up pipeline**

In [11]:
# Loads a knowledge base from given text files. This base will inform the research agent.
text_source = TextFileKnowledgeSource(
    file_paths=["g1.txt", "g2.txt", "marshal.txt", "thesis.txt"],
)
# Define the agents, each with a specialized role and backstory for context-aware processing.
research_agent = Agent(
    role="Research Agent",
    goal="Analyze the provided knowledge_base and extract historical and cultural details on the Gandhara "
         "civilization to inform design of user prompt: {prompt}.",
    backstory="You are a domain expert on ancient Gandhara and Taxila, familiar with archaeological sources like Sir "
              "John Marshall and museum archives.",
    knowledge_sources=[text_source],
    verbose=True,
    memory=False,
)

prompt_refiner_agent = Agent(
    role="Prompt Engineer",
    goal="Transform the user's raw description into a culturally informed, high-quality prompt suitable for image "
         "generation by applying best practices and avoiding common pitfalls.",
    backstory="You are a prompt engineering specialist skilled at designing text prompts for artistic AI models.",
    verbose=True,
    memory=False,
)

avatar_generator_agent = Agent(
    role="Avatar Generator",
    goal="Generate one or more artistic avatars based on the refined prompt, using a text-to-image model.",
    backstory="You are an AI image generation system that creates avatars in the Greco-Buddhist style of Gandhara.",
    verbose=True,
    memory=False,
)

backstory_generator_agent = Agent(
    role="Storyteller",
    goal=(
        "Take a refined avatar prompt and spin up an emotionally engaging, "
        "culturally rich backstory that brings the character to life."
    ),
    backstory="You are a master storyteller and world‑builder.",
    tool=BackstoryGeneratorTool(),  # singular, not a list
    verbose=True,
    memory=False,
)

research_task = Task(
    description="Research historical and cultural context on Gandhara using the provided knowledge base and initial "
                "user prompt: {prompt}.",
    expected_output="A summary of key Gandharan artistic and cultural elements.",
    agent=research_agent,
)

refine_prompt_task = Task(
    description="Refine the user prompt into a detailed, art-style-aware prompt using the research summary, "
                "good prompts are: {good_prompts}\n, and bad_prompts are: {bad_prompts}.\n"
                "Make sure the refined prompt should focus on making avatar not the environment.",
    expected_output="A refined image-generation prompt.",
    agent=prompt_refiner_agent,
)

generate_avatar_task = Task(
    description="Generate Gandharan-style avatars based on the refined prompt using the Dalle2 text-to-image tool provided.",
    expected_output="URLs or binary blobs of generated avatar images.",
    agent=avatar_generator_agent,
    tools=[DalleImageTool()]
)


generate_backstory_task = Task(
    description=(
        "Feed the refined avatar prompt into the backstory generator tool "
        "to produce a standalone narrative of ~200–300 words."
    ),
    expected_output="A compelling backstory for the generated avatar.",
    agent=backstory_generator_agent,
)
# Crew: Orchestrates the agents and tasks as a pipeline (sequential execution).
crew = Crew(
    agents=[research_agent, prompt_refiner_agent, avatar_generator_agent,backstory_generator_agent],
    tasks=[research_task, refine_prompt_task, generate_avatar_task,generate_backstory_task],
    process=Process.sequential,
)

# Utility function to read sample prompt files (good and bad examples for reference).
def read_prompts(prompts_path):
    with open(prompts_path, "r", encoding="utf-8") as f:
        prompts = f.read()

    return prompts




# **6. Implementation**

In [13]:
path = "knowledge"
good_prompts = read_prompts(os.path.join(path, "good_prompts.txt"))
bad_prompts = read_prompts(os.path.join(path, "bad_prompts.txt"))
# Prepare initial user input and reference prompt examples.
inputs = {
    "prompt": "Gandharan warrior, realistic avatar.",
    "good_prompts": good_prompts,
    "bad_prompts": bad_prompts
}
# Run the full pipeline: research, refine prompt, generate avatar, generate backstory.
result = crew.kickoff(inputs=inputs)
# Output the end result of the pipeline (likely includes image URLs and the avatar's backstory).
print(result)


# Agent: Research Agent
## Task: Research historical and cultural context on Gandhara using the provided knowledge base and initial user prompt: Gandharan warrior, realistic avatar..


# Agent: Research Agent
## Final Answer: 
The Gandhara civilization, flourishing from approximately the 5th century BCE to the 5th century CE, is renowned for its unique blend of artistic and cultural elements that reflect a fusion of Hellenistic, Persian, Central Asian, and indigenous Indian influences. This cultural synthesis is vividly illustrated in their sculptures, architecture, and artifacts, especially in the region of Taxila and other sites across ancient Gandhara.

1. **Artistic Style**: Gandharan art is characterized by its realistic portrayals of figures, drapery, and expressions, often depicting Buddhist themes. The artists focused on human anatomy, showcasing detailed musculature and lifelike features. This was a departure from earlier Indian art, which was more stylized. Notably, the figur